In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext


# New API
spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("mitraroknipartB")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",4)\
        .config("spark.cores.max",4)\
        .getOrCreate()
# Old API (RDD)
spark_context = spark_session.sparkContext

In [2]:
# 1 loead the dataset
data_frame = spark_session.read\
    .option("header", "true")\
    .csv('hdfs://192.168.2.113:9000/parking-citations.csv')\
    .cache()

In [3]:
# 1 call show
data_frame.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|null|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|         50|    99999|    99999|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|         

In [4]:
# 2 the schema:
data_frame.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: string (nullable = true)
 |-- Issue time: string (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: string (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: string (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [7]:
# 3 the numbers of rows:
data_frame.count()

9257460

In [8]:
# 4 the numbers of partitions: 
data_frame.rdd.getNumPartitions()

10

In [9]:
# 5 droping three columns from dataframe second way 
columns_to_drop = ['VIN', 'Latitude','Longitude']
data_frame2 = data_frame.drop(*columns_to_drop)
data_frame2.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|         50|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|            CA|           201512| GMC|        VN|   WH|       525 S MAIN ST| 1C51|     1|        4000A1|   NO

In [10]:
# 6 Find the maximum fine amount: 
# steps: changing the name of column 'Fine amount'  to 'Fine' because of the spece between the words 
#        converting the string to float : it was not possible by using the udf function . weird
#        


data_frame3= data_frame2.withColumnRenamed('Fine amount', 'Fine')
data_frame3.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+----+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+----+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|  50|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|            CA|           201512| GMC|        VN|   WH|       525 S MAIN ST| 1C51|     1|        4000A1|   NO EVIDENCE OF REG|  50|
|   1

In [11]:
# continuing ...  6
#data_frame3.filter("Fine amount is null").show
data_frame4=data_frame3.filter(data_frame3['Fine'].isNotNull())
data_frame4.show(10)

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+------------------+-----+------+--------------+---------------------+----+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|          Location|Route|Agency|Violation code|Violation Description|Fine|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+------------------+-----+------+--------------+---------------------+----+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|HOND|        PA|   GY|   13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|  50|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|            CA|           201512| GMC|        VN|   WH|     525 S MAIN ST| 1C51|     1|        4000A1|   NO EVIDENCE OF REG|  50|
|   1104803000|

In [12]:
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def float_of_fine(fine):
    return float(fine)

# User-defined function. Input type is a string.
udf_float_of_fine = udf(float_of_fine, StringType())


# 9999: missing (with scale factor of 10)
data_frame_with_float_of_fine = data_frame4.withColumn("floatfine", udf_float_of_fine("Fine"))


#data_frame_with_wnd_speed.select('WND', 'WND_SPEED_MS').show()

# data_frame_with_wnd_speed.show()

#data_frame_with_wnd_speed = data_frame_with_wnd_speed.filter(data_frame_with_wnd_speed['WND_SPEED_MS'] <= 900)

data_frame_with_float_of_fine.cache()

data_frame_with_float_of_fine.show()

#data_frame_with_float_of_fine.select('floatfine').summary().show()

data_frame_with_float_of_fine.printSchema()

#data_frame_with_float_of_fine.show()
# این متند دوباره یه ستون به نام فلوت فاین میده که باز استرینگ هست و به فلوت تنبدیل نشده ؟ و دوباره برا ماکس گرفتن و غیره دچار مشکل میشیم پش من یه کد دیگه رو استفاده کردم که کارمو راه انداخت هر چند یه دیتافریم دیگه ایجاد کرده و دلیلشو نمیدونم چیه ! 


Py4JJavaError: An error occurred while calling o93.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 4 times, most recent failure: Lost task 0.3 in stage 11.0 (TID 61, 192.168.2.158, executor 0): java.io.IOException: No space left on device
	at java.io.FileOutputStream.writeBytes(Native Method)
	at java.io.FileOutputStream.write(FileOutputStream.java:326)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:126)
	at java.io.DataOutputStream.write(DataOutputStream.java:107)
	at java.io.FilterOutputStream.write(FilterOutputStream.java:97)
	at org.apache.spark.sql.execution.python.DiskRowQueue.add(RowQueue.scala:130)
	at org.apache.spark.sql.execution.python.HybridRowQueue.add(RowQueue.scala:249)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$10(EvalPythonExec.scala:123)
	at scala.collection.Iterator$$anon$10.next(It

In [17]:
# continuing ...  6 
from pyspark.sql.functions import col
data_frame5 = data_frame4.select(col('Fine'), data_frame4.Fine.cast('float').alias('Finefloat'))
data_frame5.cache()
data_frame5.show()

data_frame5.summary().show()

+----+---------+
|Fine|Finefloat|
+----+---------+
|  50|     50.0|
|  50|     50.0|
|  58|     58.0|
|  93|     93.0|
|  50|     50.0|
| 163|    163.0|
| 163|    163.0|
|  93|     93.0|
|  93|     93.0|
|  93|     93.0|
|  93|     93.0|
|  50|     50.0|
|  93|     93.0|
|  68|     68.0|
|  68|     68.0|
|  68|     68.0|
|  50|     50.0|
|  93|     93.0|
|  73|     73.0|
|  73|     73.0|
+----+---------+
only showing top 20 rows

+-------+------------------+------------------+
|summary|              Fine|         Finefloat|
+-------+------------------+------------------+
|  count|           9250556|           9250556|
|   mean|  70.1290350547578|  70.1290350547578|
| stddev|32.135865052383785|32.135865052383785|
|    min|                10|              10.0|
|    25%|              63.0|              63.0|
|    50%|              68.0|              68.0|
|    75%|              73.0|              73.0|
|    max|              98.0|             505.0|
+-------+------------------+----------

In [ ]:
# still 6
#which cars ot max fine?
frequencies= data_frame4.filter(data_frame4['Fine'] =='505')
frequencies.show()
frequencies.count()


# number of max fines 
data_frame5.filter('Finefloat="505.0"').count()

In [19]:
# 7 the top 20 most frequent vehicle makes, and their frequencies.
data_frame6 = data_frame4.groupby('Make').count()
data_frame6.orderBy(("count"), ascending=False).show()


+----+-------+
|Make|  count|
+----+-------+
|TOYT|1531915|
|HOND|1042680|
|FORD| 806768|
|NISS| 661667|
|CHEV| 630841|
| BMW| 422727|
|MERZ| 376723|
|VOLK| 315865|
|HYUN| 284972|
|DODG| 271330|
|LEXS| 263248|
| KIA| 217670|
|JEEP| 214857|
|AUDI| 179652|
|MAZD| 169743|
|OTHR| 154357|
| GMC| 132662|
|INFI| 120262|
|CHRY| 120241|
|ACUR| 111204|
+----+-------+
only showing top 20 rows



In [20]:
# 8 mapping the color to dictionary & ... . 
COLORS={'AL':'Aluminum','AM':'Amber','BG':'Beige','BK':'Black','BL':'Blue','BN':'Brown','BR':'Brown','BZ':'Bronze','CH':'Charcoal','DK':'Dark','GD':'Gold','GO':'Gold','GN':'Green','GY':'Gray','GT':'Granite','IV':'Ivory','LT':'Light','OL':'Olive','OR':'Orange','MR':'Maroon','PK':'Pink','RD':'Red','RE':'Red','SI':'Silver','SL':'Silver','SM':'Smoke','TN':'Tan','VT':'Violet','WT':'White','WH':'White','YL':'Yellow','YE':'Yellow','UN':'Unknown'}
   

In [21]:
# 8 ... . 
def color_mapping(color):
    if color in COLORS:
        return COLORS[color]
    return color

udf_with_color_mapping = udf(color_mapping, StringType())
data_frame_with_long_color = data_frame4.withColumn("Long_color", udf_with_color_mapping("Color"))
data_frame_with_long_color.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+----+----------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine|Long_color|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+----+----------+
|   1103341116|2015-12-21T00:00:00|      1251|    null|       null|            CA|           200304|HOND|        PA|   GY|     13147 WELBY WAY|01521|     1|        4000A1|   NO EVIDENCE OF REG|  50|      Gray|
|   1103700150|2015-12-21T00:00:00|      1435|    null|       null|            CA|           201512| GMC|        VN|   WH|       525 S MAIN ST| 1C51|     1|    

In [25]:
#  9 the most frequent colour value for Toyotas(TOYT): 
data_frame7= data_frame_with_long_color.filter(data_frame_with_long_color['Make']=='TOYT')
data_frame7.show()

max_TOYT_color= data_frame7. select('TOYT'),
 


+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+----+----------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine|Long_color|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+----+----------+
|   4269481495|2015-12-30T00:00:00|      1711|    null|       null|            CA|           201604|TOYT|        PA|   GY|      2001 FLOWER ST|00500|    55|      80.69AP+|     NO STOP/STANDING|  93|      Gray|
|   4269951431|2015-12-30T00:00:00|      2108|    null|       null|            CA|           201609|TOYT|        PA|   GY|    3736 CIMARRON ST|00901|    55|    

AnalysisException: cannot resolve '`TOYT`' given input columns: [Agency, Body Style, Color, Fine, Issue Date, Issue time, Location, Long_color, Make, Marked Time, Meter Id, Plate Expiry Date, RP State Plate, Route, Ticket number, Violation Description, Violation code];;
'Project ['TOYT]
+- Filter (Make#24 = TOYT)
   +- Project [Ticket number#16, Issue Date#17, Issue time#18, Meter Id#19, Marked Time#20, RP State Plate#21, Plate Expiry Date#22, Make#24, Body Style#25, Color#26, Location#27, Route#28, Agency#29, Violation code#30, Violation Description#31, Fine#2108, color_mapping(Color#26) AS Long_color#5759]
      +- Filter isnotnull(Fine#2108)
         +- Project [Ticket number#16, Issue Date#17, Issue time#18, Meter Id#19, Marked Time#20, RP State Plate#21, Plate Expiry Date#22, Make#24, Body Style#25, Color#26, Location#27, Route#28, Agency#29, Violation code#30, Violation Description#31, Fine amount#32 AS Fine#2108]
            +- Project [Ticket number#16, Issue Date#17, Issue tim

In [26]:
# 9 the most frequent colour value for Toyotas(TOYT): 
max_TOYT_color = data_frame7.groupBy("Long_color")\
                     .count()\
                     .orderBy("count",ascending=False )\


max_TOYT_color.show()

+----------+------+
|Long_color| count|
+----------+------+
|      Gray|346817|
|     White|304607|
|     Black|252199|
|    Silver|248684|
|      Blue|128043|
|       Red| 84174|
|     Green| 57625|
|      Gold| 30154|
|    Maroon| 19882|
|       Tan| 17006|
|     Beige| 11572|
|        OT| 10804|
|     Brown|  8466|
|    Yellow|  3412|
|        PR|  3010|
|    Orange|  2526|
|   Unknown|  1343|
|        TU|  1077|
|        CO|   423|
|      Pink|    89|
+----------+------+
only showing top 20 rows



In [27]:
spark_context.stop()